<a href="https://colab.research.google.com/github/lucasvazelle/linux_projet_mosef/blob/main/analyse_exploratoire_VF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import pyarrow
from pyspark.sql import SparkSession
from google.colab import drive
import numpy as np


pd.set_option('display.max_columns', None)

In [4]:
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Si vous voulez utiliser les même chemin que moi, créez un raccourcit dans votre "mon drive" de mon dossier partagé "nexialog"
https://drive.google.com/drive/folders/1d-uDDNs_H3Y977kY4F9A0elEoUKUxyz6?usp=drive_link

In [5]:
chemin_donnees_brutes = "/content/drive/MyDrive/nexialog/250327_tests_fixe_dns_sah_202412_202501_NOUVEAU.parquet"

In [6]:
chemin_donnees_preprocessees = "/content/drive/MyDrive/nexialog/data_preprocessed_FINAL.parquet"

In [7]:
donnee_brutes = pd.read_parquet(chemin_donnees_brutes, engine="pyarrow").sort_values(by="date_hour")
donnee_brutes.head()

,date_hour,code_departement,olt_model,olt_name,peag_nro,boucle,dsp,pebib,pop_dns,nb_test_dns,avg_dns_time,std_dns_time,nb_test_scoring,avg_latence_scoring,std_latence_scoring,avg_score_scoring,std_score_scoring,nb_client_total
0,2024-12-01,01,M11,01_olt_3,01_peag_3,BU1464,dsp_1,69_pebib_3,69_lyon,23,6.845641,0.377309,9,11.437500,0.281250,3.258823,0.375156,32
3812,2024-12-01,64,M22,64_olt_3776,64_peag_1887,BU124,dsp_25,None,33_bdx,8,5.327906,0.300715,3,10.500000,0.324760,3.452941,0.005094,11
3811,2024-12-01,64,M22,64_olt_3775,64_peag_1886,BU134,dsp_25,None,33_bdx,5,5.758650,0.338825,3,10.687500,0.000000,3.450000,0.000000,8
3810,2024-12-01,64,M22,64_olt_3774,64_peag_1885,BU112,dsp_25,None,33_bdx,17,3.583456,0.252478,4,9.140625,0.281250,3.467647,0.000000,21
3809,2024-12-01,64,M22,64_olt_3773,64_peag_1884,BU124,dsp_25,None,33_bdx,10,3.638700,0.616376,10,9.112500,0.237171,3.465882,0.005581,20


# rapport des données

In [8]:
! pip install ydata-profiling

In [9]:
#from ydata_profiling import ProfileReport
#profil = ProfileReport(donnee_brutes)  # explorative=True active toutes les analyses

In [10]:
# profil.to_file("rapport_données_brutes.html")

# EDA

In [11]:
donnee_brutes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9096908 entries, 0 to 9096907
Data columns (total 18 columns):
 #   Column               Dtype         
---  ------               -----         
 0   date_hour            datetime64[ns]
 1   code_departement     object        
 2   olt_model            object        
 3   olt_name             object        
 4   peag_nro             object        
 5   boucle               object        
 6   dsp                  object        
 7   pebib                object        
 8   pop_dns              object        
 9   nb_test_dns          int64         
 10  avg_dns_time         float64       
 11  std_dns_time         float64       
 12  nb_test_scoring      int64         
 13  avg_latence_scoring  float64       
 14  std_latence_scoring  float64       
 15  avg_score_scoring    float64       
 16  std_score_scoring    float64       
 17  nb_client_total      int64         
dtypes: datetime64[ns](1), float64(6), int64(3), object(8)
memory usage:

In [12]:
donnee_brutes.describe()

,date_hour,nb_test_dns,avg_dns_time,std_dns_time,nb_test_scoring,avg_latence_scoring,std_latence_scoring,avg_score_scoring,std_score_scoring,nb_client_total
count,9096908,9.096908e+06,8.379976e+06,8.239312e+06,9.096908e+06,8.313430e+06,6.949929e+06,8.313409e+06,6.949911e+06,9.096908e+06
mean,2024-12-31 08:27:32.510107136,1.649363e+01,5.407638e+00,6.848560e-01,8.786119e+00,9.440936e+00,5.834277e-01,3.245417e+00,2.739184e-01,2.527975e+01
min,2024-12-01 00:00:00,0.000000e+00,6.637500e-01,0.000000e+00,0.000000e+00,3.375000e+00,0.000000e+00,7.500000e-01,0.000000e+00,1.000000e+00
25%,2024-12-15 19:00:00,3.000000e+00,3.287672e+00,3.098163e-01,2.000000e+00,7.643382e+00,2.812500e-01,3.111555e+00,4.556381e-03,5.000000e+00
50%,2024-12-30 15:00:00,8.000000e+00,4.660167e+00,4.406154e-01,4.000000e+00,9.337500e+00,3.557562e-01,3.432353e+00,5.094272e-02,1.200000e+01
75%,2025-01-15 22:00:00,1.800000e+01,6.586500e+00,6.933917e-01,1.000000e+01,1.106250e+01,5.031153e-01,3.467647e+00,4.741837e-01,2.800000e+01
max,2025-01-31 23:00:00,1.275000e+03,3.239557e+02,3.675479e+02,2.050000e+02,2.129625e+03,1.507031e+03,3.564706e+00,1.984058e+00,1.330000e+03
std,NaN,2.601294e+01,3.454416e+00,2.241174e+00,1.322018e+01,3.540929e+00,2.693535e+00,3.614666e-01,3.298146e-01,3.856525e+01


In [13]:
len(donnee_brutes)

9096908

In [14]:
len(donnee_brutes.drop_duplicates(["date_hour","olt_name"])) # supprime les lignes en double dans le df en ne regardant que les colonnes date_hour et olt_name

8492270

In [15]:
len(donnee_brutes.drop_duplicates(["date_hour","peag_nro"]))

4447318

In [16]:
#
len(donnee_brutes[donnee_brutes['nb_client_total'] < 10])


3813944

# Premiers traitements


### BOX → PM → BOUCLE → PAEG → OLT → PEU → PEBIB → SERVEUR

In [17]:
donnee_brutes.columns

Index(['date_hour', 'code_departement', 'olt_model', 'olt_name', 'peag_nro',
       'boucle', 'dsp', 'pebib', 'pop_dns', 'nb_test_dns', 'avg_dns_time',
       'std_dns_time', 'nb_test_scoring', 'avg_latence_scoring',
       'std_latence_scoring', 'avg_score_scoring', 'std_score_scoring',
       'nb_client_total'],
      dtype='object')

In [18]:
# ____________________________________________________________calcul feature pour avoir le moment de la journée
def definir_moment_journee(heure):
    if 0 <= heure < 6:
        return "nuit"
    elif 6 <= heure < 12:
        return "matin"
    elif 12 <= heure < 18:
        return "après-midi"
    else:
        return "soir"

donnee_brutes["moment_journee"] = donnee_brutes["date_hour"].dt.hour.apply(definir_moment_journee)

print(donnee_brutes[["date_hour", "moment_journee"]].head())

      date_hour moment_journee
0    2024-12-01           nuit
3812 2024-12-01           nuit
3811 2024-12-01           nuit
3810 2024-12-01           nuit
3809 2024-12-01           nuit


In [19]:
donnee_brutes.head(2)

,date_hour,code_departement,olt_model,olt_name,peag_nro,boucle,dsp,pebib,pop_dns,nb_test_dns,avg_dns_time,std_dns_time,nb_test_scoring,avg_latence_scoring,std_latence_scoring,avg_score_scoring,std_score_scoring,nb_client_total,moment_journee
0,2024-12-01,01,M11,01_olt_3,01_peag_3,BU1464,dsp_1,69_pebib_3,69_lyon,23,6.845641,0.377309,9,11.4375,0.28125,3.258823,0.375156,32,nuit
3812,2024-12-01,64,M22,64_olt_3776,64_peag_1887,BU124,dsp_25,None,33_bdx,8,5.327906,0.300715,3,10.5000,0.32476,3.452941,0.005094,11,nuit


In [20]:
#_____________________________________________ création features si le test dns est au même département que le PM_______________________________

departement_region = {
    # Auvergne-Rhône-Alpes
    "01": "Auvergne-Rhône-Alpes", "03": "Auvergne-Rhône-Alpes", "07": "Auvergne-Rhône-Alpes",
    "15": "Auvergne-Rhône-Alpes", "26": "Auvergne-Rhône-Alpes", "38": "Auvergne-Rhône-Alpes",
    "42": "Auvergne-Rhône-Alpes", "43": "Auvergne-Rhône-Alpes", "63": "Auvergne-Rhône-Alpes",
    "69": "Auvergne-Rhône-Alpes", "73": "Auvergne-Rhône-Alpes", "74": "Auvergne-Rhône-Alpes",
    # Bourgogne-Franche-Comté
    "21": "Bourgogne-Franche-Comté", "25": "Bourgogne-Franche-Comté", "39": "Bourgogne-Franche-Comté",
    "58": "Bourgogne-Franche-Comté", "70": "Bourgogne-Franche-Comté", "71": "Bourgogne-Franche-Comté",
    "89": "Bourgogne-Franche-Comté", "90": "Bourgogne-Franche-Comté",
    # Bretagne
    "22": "Bretagne", "29": "Bretagne", "35": "Bretagne", "56": "Bretagne",
    # Centre-Val de Loire
    "18": "Centre-Val de Loire", "28": "Centre-Val de Loire", "36": "Centre-Val de Loire",
    "37": "Centre-Val de Loire", "41": "Centre-Val de Loire", "45": "Centre-Val de Loire",
    # Corse
    "2A": "Corse", "2B": "Corse",
    # Grand Est
    "08": "Grand Est", "10": "Grand Est", "51": "Grand Est", "52": "Grand Est",
    "54": "Grand Est", "55": "Grand Est", "57": "Grand Est", "67": "Grand Est",
    "68": "Grand Est", "88": "Grand Est",
    # Hauts-de-France
    "02": "Hauts-de-France", "59": "Hauts-de-France", "60": "Hauts-de-France",
    "62": "Hauts-de-France", "80": "Hauts-de-France",
    # Île-de-France
    "75": "Île-de-France", "77": "Île-de-France", "78": "Île-de-France",
    "91": "Île-de-France", "92": "Île-de-France", "93": "Île-de-France",
    "94": "Île-de-France", "95": "Île-de-France",
    # Normandie
    "14": "Normandie", "27": "Normandie", "50": "Normandie",
    "61": "Normandie", "76": "Normandie",
    # Nouvelle-Aquitaine
    "16": "Nouvelle-Aquitaine", "17": "Nouvelle-Aquitaine", "19": "Nouvelle-Aquitaine",
    "23": "Nouvelle-Aquitaine", "24": "Nouvelle-Aquitaine", "33": "Nouvelle-Aquitaine",
    "40": "Nouvelle-Aquitaine", "47": "Nouvelle-Aquitaine", "64": "Nouvelle-Aquitaine",
    "79": "Nouvelle-Aquitaine", "86": "Nouvelle-Aquitaine", "87": "Nouvelle-Aquitaine",
    # Occitanie
    "09": "Occitanie", "11": "Occitanie", "12": "Occitanie", "30": "Occitanie",
    "31": "Occitanie", "32": "Occitanie", "34": "Occitanie", "46": "Occitanie",
    "48": "Occitanie", "65": "Occitanie", "66": "Occitanie", "81": "Occitanie",
    "82": "Occitanie",
    # Pays de la Loire
    "44": "Pays de la Loire", "49": "Pays de la Loire", "53": "Pays de la Loire",
    "72": "Pays de la Loire", "85": "Pays de la Loire",
    # Provence-Alpes-Côte d'Azur
    "04": "Provence-Alpes-Côte d'Azur", "05": "Provence-Alpes-Côte d'Azur",
    "06": "Provence-Alpes-Côte d'Azur", "13": "Provence-Alpes-Côte d'Azur",
    "83": "Provence-Alpes-Côte d'Azur", "84": "Provence-Alpes-Côte d'Azur",
    # Départements d'outre-mer
    "971": "Guadeloupe", "972": "Martinique", "973": "Guyane", "974": "La Réunion", "976": "Mayotte"
}

donnee_brutes["dep_pop_dns"] = donnee_brutes["pop_dns"].str[:2]

# Vérifier si les départements sont égaux
donnee_brutes["depts_egaux"] = donnee_brutes["dep_pop_dns"] == donnee_brutes["code_departement"]

# Vérifier si les départements appartiennent à la même région
donnee_brutes["region_pop_dns"] = donnee_brutes["dep_pop_dns"].map(departement_region)
donnee_brutes["region_pm"] = donnee_brutes["code_departement"].map(departement_region)
donnee_brutes["meme_region"] = donnee_brutes["region_pop_dns"] == donnee_brutes["region_pm"]

# Afficher un aperçu
print(donnee_brutes[["code_departement", "pop_dns", "dep_pop_dns", "depts_egaux", "meme_region"]].head())

     code_departement  pop_dns dep_pop_dns  depts_egaux  meme_region
0                  01  69_lyon          69        False         True
3812               64   33_bdx          33        False         True
3811               64   33_bdx          33        False         True
3810               64   33_bdx          33        False         True
3809               64   33_bdx          33        False         True


In [21]:
donnee_brutes.head(2)

,date_hour,code_departement,olt_model,olt_name,peag_nro,boucle,dsp,pebib,pop_dns,nb_test_dns,avg_dns_time,std_dns_time,nb_test_scoring,avg_latence_scoring,std_latence_scoring,avg_score_scoring,std_score_scoring,nb_client_total,moment_journee,dep_pop_dns,depts_egaux,region_pop_dns,region_pm,meme_region
0,2024-12-01,01,M11,01_olt_3,01_peag_3,BU1464,dsp_1,69_pebib_3,69_lyon,23,6.845641,0.377309,9,11.4375,0.28125,3.258823,0.375156,32,nuit,69,False,Auvergne-Rhône-Alpes,Auvergne-Rhône-Alpes,True
3812,2024-12-01,64,M22,64_olt_3776,64_peag_1887,BU124,dsp_25,None,33_bdx,8,5.327906,0.300715,3,10.5000,0.32476,3.452941,0.005094,11,nuit,33,False,Nouvelle-Aquitaine,Nouvelle-Aquitaine,True


In [22]:
#___________________________________ supression ligne si pas beaucoup de client au niveau PM (en moyenne il y a 25 clients par PM)

donnee_brutes_sans_pm_avec_peu_de_clients = donnee_brutes[donnee_brutes['nb_client_total'] > 15]
assert len(donnee_brutes_sans_pm_avec_peu_de_clients.drop_duplicates(["date_hour"])) ==  len(donnee_brutes.drop_duplicates(["date_hour"]))

### on supprime les PM qui ont 15 clients ou moins

In [23]:
# _________________________________________supression ligne si pas beaucoup de test au niveau des PM

seuil_nombre_de_test_minimum = 5

# Suppression des lignes où les deux colonnes sont en dessous du seuil
donnee_brutes_sans_pm_avec_peu_de_clients = donnee_brutes_sans_pm_avec_peu_de_clients[
    ~((donnee_brutes_sans_pm_avec_peu_de_clients["nb_test_scoring"] < seuil_nombre_de_test_minimum) &
      (donnee_brutes_sans_pm_avec_peu_de_clients["nb_test_dns"] < seuil_nombre_de_test_minimum))
]
donnee_brutes_sans_pm_avec_peu_de_clients.loc[
    donnee_brutes_sans_pm_avec_peu_de_clients["nb_test_dns"] < seuil_nombre_de_test_minimum,
    ["avg_dns_time", "std_dns_time"]
] = np.nan

# Remplacer par NaN si nb_test_scoring est sous le seuil
donnee_brutes_sans_pm_avec_peu_de_clients.loc[
    donnee_brutes_sans_pm_avec_peu_de_clients["nb_test_scoring"] < seuil_nombre_de_test_minimum,
    [ "avg_latence_scoring", "std_latence_scoring", "avg_score_scoring", "std_score_scoring"]
] = np.nan

donnee_brutes_sans_pm_avec_peu_de_clients_et_peu_de_tests = donnee_brutes_sans_pm_avec_peu_de_clients

# Afficher un aperçu
print(donnee_brutes_sans_pm_avec_peu_de_clients_et_peu_de_tests[
    ["nb_test_scoring", "nb_test_dns", "avg_dns_time", "std_dns_time",
     "avg_latence_scoring", "std_latence_scoring", "avg_score_scoring", "std_score_scoring"]
].head())

assert len(donnee_brutes_sans_pm_avec_peu_de_clients_et_peu_de_tests.drop_duplicates(["date_hour"])) ==  len(donnee_brutes.drop_duplicates(["date_hour"]))

      nb_test_scoring  nb_test_dns  avg_dns_time  std_dns_time  \
0                   9           23      6.845641      0.377309   
3810                4           17      3.583456      0.252478   
3809               10           10      3.638700      0.616376   
3802                5           14      4.508839      0.534071   
3801               12           20      4.808550      0.810622   

      avg_latence_scoring  std_latence_scoring  avg_score_scoring  \
0               11.437500             0.281250           3.258823   
3810                  NaN                  NaN                NaN   
3809             9.112500             0.237171           3.465882   
3802             9.675000             0.251558           3.465882   
3801             9.796875             0.289647           3.404412   

      std_score_scoring  
0              0.375156  
3810                NaN  
3809           0.005581  
3802           0.003946  
3801           0.207982  


In [24]:
donnee_brutes_sans_pm_avec_peu_de_clients_et_peu_de_tests.head()

,date_hour,code_departement,olt_model,olt_name,peag_nro,boucle,dsp,pebib,pop_dns,nb_test_dns,avg_dns_time,std_dns_time,nb_test_scoring,avg_latence_scoring,std_latence_scoring,avg_score_scoring,std_score_scoring,nb_client_total,moment_journee,dep_pop_dns,depts_egaux,region_pop_dns,region_pm,meme_region
0,2024-12-01,01,M11,01_olt_3,01_peag_3,BU1464,dsp_1,69_pebib_3,69_lyon,23,6.845641,0.377309,9,11.437500,0.281250,3.258823,0.375156,32,nuit,69,False,Auvergne-Rhône-Alpes,Auvergne-Rhône-Alpes,True
3810,2024-12-01,64,M22,64_olt_3774,64_peag_1885,BU112,dsp_25,None,33_bdx,17,3.583456,0.252478,4,NaN,NaN,NaN,NaN,21,nuit,33,False,Nouvelle-Aquitaine,Nouvelle-Aquitaine,True
3809,2024-12-01,64,M22,64_olt_3773,64_peag_1884,BU124,dsp_25,None,33_bdx,10,3.638700,0.616376,10,9.112500,0.237171,3.465882,0.005581,20,nuit,33,False,Nouvelle-Aquitaine,Nouvelle-Aquitaine,True
3802,2024-12-01,64,M22,64_olt_3766,64_peag_1877,BU124,dsp_25,None,33_bdx,14,4.508839,0.534071,5,9.675000,0.251558,3.465882,0.003946,19,nuit,33,False,Nouvelle-Aquitaine,Nouvelle-Aquitaine,True
3801,2024-12-01,64,M22,64_olt_3765,64_peag_1876,BU136,dsp_25,None,33_bdx,20,4.808550,0.810622,12,9.796875,0.289647,3.404412,0.207982,32,nuit,33,False,Nouvelle-Aquitaine,Nouvelle-Aquitaine,True


In [25]:
donnee_brutes_sans_pm_avec_peu_de_clients_et_peu_de_tests.duplicated().sum()

np.int64(0)

In [26]:
donnee_brutes_sans_pm_avec_peu_de_clients_et_peu_de_tests.shape

(3891007, 24)

In [27]:
donnee_brutes_sans_pm_avec_peu_de_clients_et_peu_de_tests = donnee_brutes_sans_pm_avec_peu_de_clients_et_peu_de_tests[['date_hour', 'code_departement', 'olt_model', 'olt_name', 'peag_nro',
 'boucle', 'dsp', 'pebib', 'pop_dns', 'nb_test_dns', 'avg_dns_time',
'std_dns_time', 'nb_test_scoring', 'avg_latence_scoring',
'std_latence_scoring', 'avg_score_scoring', 'std_score_scoring',
'nb_client_total', 'moment_journee', 'depts_egaux', 'meme_region']]
donnee_brutes_sans_pm_avec_peu_de_clients_et_peu_de_tests.head()

,date_hour,code_departement,olt_model,olt_name,peag_nro,boucle,dsp,pebib,pop_dns,nb_test_dns,avg_dns_time,std_dns_time,nb_test_scoring,avg_latence_scoring,std_latence_scoring,avg_score_scoring,std_score_scoring,nb_client_total,moment_journee,depts_egaux,meme_region
0,2024-12-01,01,M11,01_olt_3,01_peag_3,BU1464,dsp_1,69_pebib_3,69_lyon,23,6.845641,0.377309,9,11.437500,0.281250,3.258823,0.375156,32,nuit,False,True
3810,2024-12-01,64,M22,64_olt_3774,64_peag_1885,BU112,dsp_25,None,33_bdx,17,3.583456,0.252478,4,NaN,NaN,NaN,NaN,21,nuit,False,True
3809,2024-12-01,64,M22,64_olt_3773,64_peag_1884,BU124,dsp_25,None,33_bdx,10,3.638700,0.616376,10,9.112500,0.237171,3.465882,0.005581,20,nuit,False,True
3802,2024-12-01,64,M22,64_olt_3766,64_peag_1877,BU124,dsp_25,None,33_bdx,14,4.508839,0.534071,5,9.675000,0.251558,3.465882,0.003946,19,nuit,False,True
3801,2024-12-01,64,M22,64_olt_3765,64_peag_1876,BU136,dsp_25,None,33_bdx,20,4.808550,0.810622,12,9.796875,0.289647,3.404412,0.207982,32,nuit,False,True


In [28]:
donnee_brutes_sans_pm_avec_peu_de_clients_et_peu_de_tests['olt_name'].value_counts()

,count
olt_name,
94_olt_5419,2976
77_olt_4726,2976
95_olt_5551,2976
93_olt_5549,2976
26_olt_288,2922
...,...
68_olt_3952,1
30_olt_1553,1
21_olt_1009,1


### Valeurs Manquantes

In [29]:
missing_values = donnee_brutes_sans_pm_avec_peu_de_clients_et_peu_de_tests.isna().sum()
missing_percentage = (donnee_brutes_sans_pm_avec_peu_de_clients_et_peu_de_tests.isna().mean() * 100).round(2)

missing_df = pd.DataFrame({'Valeurs Manquantes': missing_values, 'Pourcentage (%)': missing_percentage})
missing_df

,Valeurs Manquantes,Pourcentage (%)
date_hour,0,0.00
code_departement,0,0.00
olt_model,3037,0.08
olt_name,0,0.00
peag_nro,0,0.00
boucle,0,0.00
dsp,0,0.00
pebib,1779424,45.73
pop_dns,2,0.00
nb_test_dns,0,0.00


In [30]:
# on va drop la colonne pepib au final parce que bcp de valeurs manquante et un niveau d'agrégation élevé
donnee_brutes_sans_pm_avec_peu_de_clients_et_peu_de_tests= donnee_brutes_sans_pm_avec_peu_de_clients_et_peu_de_tests.drop('pebib', axis=1)
donnee_brutes_sans_pm_avec_peu_de_clients_et_peu_de_tests.head(2)

,date_hour,code_departement,olt_model,olt_name,peag_nro,boucle,dsp,pop_dns,nb_test_dns,avg_dns_time,std_dns_time,nb_test_scoring,avg_latence_scoring,std_latence_scoring,avg_score_scoring,std_score_scoring,nb_client_total,moment_journee,depts_egaux,meme_region
0,2024-12-01,01,M11,01_olt_3,01_peag_3,BU1464,dsp_1,69_lyon,23,6.845641,0.377309,9,11.4375,0.28125,3.258823,0.375156,32,nuit,False,True
3810,2024-12-01,64,M22,64_olt_3774,64_peag_1885,BU112,dsp_25,33_bdx,17,3.583456,0.252478,4,NaN,NaN,NaN,NaN,21,nuit,False,True


#### **olt_model** (0.08% manquantes)




**Pour traiter les valeurs manquantes de la variable olt_model j'ai fait une imputation basée sur la géographie du réseau. Après avoir identifié que les 3334 valeurs manquantes sont concentrées sur seulement 5 OLTs spécifiques et qu'aucun de ces équipements n'a de modèle connu dans le dataset, j'ai analysé la distribution des modèles par département -> par exemple le département 35 privilégie le modèle "M11" tandis que les départements 26, 24 et 07 favorisent le modèle "M22" dcp j'ai imputé chaque valeur manquante en utilisant le modèle le plus fréquent dans le département correspondant..**

In [31]:
donnee_brutes_sans_pm_avec_peu_de_clients_et_peu_de_tests.olt_model.value_counts()

,count
olt_model,
M22,1638806
M13,747777
M11,687089
old0,367094
M24,277872
old1,133669
M20,35663


In [32]:
print(f"Nombre de valeurs manquantes pour olt_model: {donnee_brutes_sans_pm_avec_peu_de_clients_et_peu_de_tests['olt_model'].isna().sum()}")

Nombre de valeurs manquantes pour olt_model: 3037


In [33]:
# Filtrer les lignes avec olt_model manquant
missing_olt_model = donnee_brutes_sans_pm_avec_peu_de_clients_et_peu_de_tests[donnee_brutes_sans_pm_avec_peu_de_clients_et_peu_de_tests['olt_model'].isna()]
missing_olt_model.head()

,date_hour,code_departement,olt_model,olt_name,peag_nro,boucle,dsp,pop_dns,nb_test_dns,avg_dns_time,std_dns_time,nb_test_scoring,avg_latence_scoring,std_latence_scoring,avg_score_scoring,std_score_scoring,nb_client_total,moment_journee,depts_egaux,meme_region
1263,2024-12-01 00:00:00,26,None,26_olt_1301,26_peag_613,BU400,dsp_1,13_abg,78,5.835010,1.271536,50,13.207500,1.219947,3.119470,0.507015,128,nuit,False,False
1983,2024-12-01 00:00:00,35,None,35_olt_2049,35_peag_1082,BU836,dsp_1,35_ren,16,1.583484,0.428641,5,7.762500,0.251558,3.487059,0.003946,21,nuit,True,True
6969,2024-12-01 01:00:00,26,None,26_olt_1301,26_peag_613,BU400,dsp_1,13_abg,94,5.615585,1.285407,44,13.078125,0.323814,3.059358,0.415081,138,nuit,False,False
7698,2024-12-01 01:00:00,35,None,35_olt_2049,35_peag_1082,BU836,dsp_1,35_ren,13,1.641462,0.337330,10,7.931250,0.319301,3.392647,0.289918,23,nuit,True,True
12720,2024-12-01 02:00:00,26,None,26_olt_1301,26_peag_613,BU400,dsp_1,13_abg,93,5.638500,1.073475,44,13.103693,0.412809,3.130949,0.513650,137,nuit,False,False


In [34]:
# nbre d'olt_name uniques concernés par ces valeurs manquantes
unique_olt_names = missing_olt_model['olt_name'].nunique()
print(f"Nombre d'olt_name uniques avec des modèles manquants: {unique_olt_names}")

Nombre d'olt_name uniques avec des modèles manquants: 5


In [35]:
olt_counts = missing_olt_model['olt_name'].value_counts()
print("\nles olt_name avec le plus de valeurs manquantes:")
print(olt_counts.head(10))


les olt_name avec le plus de valeurs manquantes:
olt_name
26_olt_1301    1488
35_olt_2049    1402
24_olt_1192     142
24_olt_1193       4
07_olt_330        1
Name: count, dtype: int64


In [36]:
# Pour chaque olt_name avec des valeurs manquantes, on vérifie s'il existe des enregistrements avec un modèle connu
olt_with_known_models = 0
olt_without_known_models = 0

for olt_name in missing_olt_model['olt_name'].unique():
    # Cherche des enregistrements pour cet olt_name avec un modèle connu
    known_models = donnee_brutes_sans_pm_avec_peu_de_clients_et_peu_de_tests[
        (donnee_brutes_sans_pm_avec_peu_de_clients_et_peu_de_tests['olt_name'] == olt_name) &
        (~donnee_brutes_sans_pm_avec_peu_de_clients_et_peu_de_tests['olt_model'].isna())
    ]['olt_model']

    if len(known_models) > 0:
        olt_with_known_models += 1
    else:
        olt_without_known_models += 1

print(f"\nolt_name avec au moins un modèle connu: {olt_with_known_models}")
print(f"olt_name sans aucun modèle connu: {olt_without_known_models}")


olt_name avec au moins un modèle connu: 0
olt_name sans aucun modèle connu: 5


* on ne peut pas utiliser le modèle le plus fréquent pour chaque OLT ici vu que aucun de ces 5 OLT manquants n'a de modèle connu dans le dataset .. on va changer la méthode d'imputation ..

In [37]:
df_imputed = donnee_brutes_sans_pm_avec_peu_de_clients_et_peu_de_tests.copy()

In [38]:
# je vais tester l'imputation basée sur le département dans ce cas ..
df_imputed['dept'] = df_imputed['olt_name'].str.extract(r'^(\d+)_')

print("Premiers départements extraits:")
print(df_imputed[['olt_name', 'dept']].head(10))

Premiers départements extraits:
         olt_name dept
0        01_olt_3   01
3810  64_olt_3774   64
3809  64_olt_3773   64
3802  64_olt_3766   64
3801  64_olt_3765   64
3799  64_olt_3763   64
3796  64_olt_3760   64
3795  64_olt_3759   64
3816  64_olt_3780   64
3834  64_olt_3798   64


In [39]:
# Pour chaque département -> la distribution des modèles
dept_model_distribution = {}
for dept in df_imputed['dept'].unique():
    if pd.isna(dept):
        continue

    # Obtenir la distribution des modèles pour ce département
    model_counts = df_imputed[
        (df_imputed['dept'] == dept) &
        (~df_imputed['olt_model'].isna())
    ]['olt_model'].value_counts()

    # Si le département a des modèles connus
    if not model_counts.empty:
        dept_model_distribution[dept] = model_counts.to_dict()

# Départements qui nous intéressent (ou on a des valeurs manquantes)
for dept in ['26', '35', '24', '07']:
    if dept in dept_model_distribution:
        print(f"Département {dept}: {dept_model_distribution[dept]}")
    else:
        print(f"Département {dept}: Aucun modèle connu")

Département 26: {'M22': 13123, 'M13': 4410, 'M11': 1569, 'old0': 1488, 'M20': 76}
Département 35: {'M11': 22167, 'M22': 19609, 'M24': 7096, 'old0': 5952, 'M13': 5680, 'M20': 166}
Département 24: {'M22': 15782, 'M13': 2976, 'M11': 380, 'M20': 279}
Département 07: {'M22': 6527, 'M13': 1488, 'M11': 103, 'M20': 64}


In [40]:
# Créer un dictionnaire des modèles les plus fréquents par département
dept_model = {}
for dept, models in dept_model_distribution.items():
    # Trouver le modèle avec le plus d'occurrences
    most_common_model = max(models.items(), key=lambda x: x[1])[0]
    dept_model[dept] = most_common_model

In [41]:
# Compter les valeurs manquantes avant imputation
missing_before = df_imputed['olt_model'].isna().sum()
print(f"Valeurs manquantes avant imputation: {missing_before}")

# Le modèle le plus fréquent globalement
default_model = 'M22'

# Imputer les valeurs manquantes
for idx in df_imputed[df_imputed['olt_model'].isna()].index:
    dept = df_imputed.loc[idx, 'dept']
    if dept in dept_model:
        # le modèle le plus fréquent du département
        df_imputed.loc[idx, 'olt_model'] = dept_model[dept]
    else:
        # le modèle par défaut
        df_imputed.loc[idx, 'olt_model'] = default_model

print(f"Valeurs manquantes finales dans olt_model: {df_imputed['olt_model'].isna().sum()}")

Valeurs manquantes avant imputation: 3037
Valeurs manquantes finales dans olt_model: 0


In [42]:
df_imputed=df_imputed.drop('dept', axis=1)

#### pop_dns (2 valeurs manquantes)

**pour l'imputation des valeurs manquantes de pop_dns j'ai cherché d'abord la valeur la plus fréquente pour la combinaison exacte d'OLT et PEAG, car les équipements partageant le même chemin réseau utilisent généralement le même serveur DNS.. Si cette information n'est pas disponible, on impute par la valeur au niveau de l'OLT seul. En dernier recours, nous utilisons la valeur la plus fréquente dans l'ensemble de la chaine technique..**

In [43]:
df_imputed.pop_dns.value_counts()

,count
pop_dns,
94_cbv,724457
77_mit,697627
33_bdx,582871
69_lyon,525353
13_abg,502237
35_ren,458096
67_sbg,249155
59_lil,151209


In [44]:
# lignes avec pop_dns manquant
missing_pop_dns = df_imputed[df_imputed['pop_dns'].isna()]
missing_pop_dns

,date_hour,code_departement,olt_model,olt_name,peag_nro,boucle,dsp,pop_dns,nb_test_dns,avg_dns_time,std_dns_time,nb_test_scoring,avg_latence_scoring,std_latence_scoring,avg_score_scoring,std_score_scoring,nb_client_total,moment_journee,depts_egaux,meme_region
5184563,2025-01-04 04:00:00,60,M13,60_olt_3511,60_peag_1702,BU334,dsp_1,None,0,NaN,NaN,17,5.525735,0.409279,3.252249,0.705607,17,nuit,False,False
6512947,2025-01-13 16:00:00,75,M13,75_olt_4505,75_peag_2301,BU270,dsp_1,None,0,NaN,NaN,20,4.865625,0.555799,3.000882,1.111764,20,après-midi,False,False


In [45]:
# Créer dictionnaires de valeurs les plus fréquentes par niveau de granularité
pop_by_olt_peag = df_imputed.dropna(subset=['pop_dns']).groupby(['olt_name', 'peag_nro'])['pop_dns'].apply(
    lambda x: x.value_counts().index[0] if len(x) > 0 else None
).to_dict()

pop_by_olt = df_imputed.dropna(subset=['pop_dns']).groupby('olt_name')['pop_dns'].apply(
    lambda x: x.value_counts().index[0] if len(x) > 0 else None
).to_dict()

default_pop = df_imputed['pop_dns'].mode()[0]

# Imputer les valeurs manquantes
for idx in missing_pop_dns.index:
    olt = df_imputed.loc[idx, 'olt_name']
    peag = df_imputed.loc[idx, 'peag_nro']

    # Chercher la valeur à imputer par ordre de spécificité
    if (olt, peag) in pop_by_olt_peag:
        df_imputed.loc[idx, 'pop_dns'] = pop_by_olt_peag[(olt, peag)]
    elif olt in pop_by_olt:
        df_imputed.loc[idx, 'pop_dns'] = pop_by_olt[olt]
    else:
        df_imputed.loc[idx, 'pop_dns'] = default_pop

In [46]:
df_imputed['pop_dns'].isna().sum()

np.int64(0)

### Traitement des valeurs manquantes des tests

In [47]:
# verif si les valeurs manquantes de avg_dns_time correspondent à nb_test_dns = 0 ???
missing_avg_dns = df_imputed['avg_dns_time'].isna()
zero_dns_tests = df_imputed['nb_test_dns'] == 0
total_missing_avg_dns = missing_avg_dns.sum()
total_zero_dns_tests = zero_dns_tests.sum()

print(f"Nombre total de valeurs manquantes pour avg_dns_time: {total_missing_avg_dns}")
print(f"Nombre total de lignes avec nb_test_dns = 0: {total_zero_dns_tests}")

Nombre total de valeurs manquantes pour avg_dns_time: 2709
Nombre total de lignes avec nb_test_dns = 0: 2


-> les nombre de valeurs manquantes ici ne correpond pas forcément à l'absence du test (c plutot le traitement qui a été fait plus haut en mettant les conditions sur le nombre de test au niveau des PM..)

* on a opté à garder les nans ici et creer une variable indicatrice pour les moyennes et variance de chaque tests pour indiquer si y a des valeurs manquantes ou pas (ça va nous aider au moment de la construction des modeles pour les OLT et pour les modèles de PEAG)

In [48]:
# Création de variables indicatrices pour les tests DNS
df_imputed['is_missing_avg_dns_time'] = df_imputed['avg_dns_time'].isna().astype(int)
df_imputed['is_missing_std_dns_time'] = df_imputed['std_dns_time'].isna().astype(int)

# Création de variables indicatrices pour les tests de scoring
df_imputed['is_missing_avg_latence_scoring'] = df_imputed['avg_latence_scoring'].isna().astype(int)
df_imputed['is_missing_std_latence_scoring'] = df_imputed['std_latence_scoring'].isna().astype(int)
df_imputed['is_missing_avg_score_scoring'] = df_imputed['avg_score_scoring'].isna().astype(int)
df_imputed['is_missing_std_score_scoring'] = df_imputed['std_score_scoring'].isna().astype(int)

In [49]:
df_imputed.head()

,date_hour,code_departement,olt_model,olt_name,peag_nro,boucle,dsp,pop_dns,nb_test_dns,avg_dns_time,std_dns_time,nb_test_scoring,avg_latence_scoring,std_latence_scoring,avg_score_scoring,std_score_scoring,nb_client_total,moment_journee,depts_egaux,meme_region,is_missing_avg_dns_time,is_missing_std_dns_time,is_missing_avg_latence_scoring,is_missing_std_latence_scoring,is_missing_avg_score_scoring,is_missing_std_score_scoring
0,2024-12-01,01,M11,01_olt_3,01_peag_3,BU1464,dsp_1,69_lyon,23,6.845641,0.377309,9,11.437500,0.281250,3.258823,0.375156,32,nuit,False,True,0,0,0,0,0,0
3810,2024-12-01,64,M22,64_olt_3774,64_peag_1885,BU112,dsp_25,33_bdx,17,3.583456,0.252478,4,NaN,NaN,NaN,NaN,21,nuit,False,True,0,0,1,1,1,1
3809,2024-12-01,64,M22,64_olt_3773,64_peag_1884,BU124,dsp_25,33_bdx,10,3.638700,0.616376,10,9.112500,0.237171,3.465882,0.005581,20,nuit,False,True,0,0,0,0,0,0
3802,2024-12-01,64,M22,64_olt_3766,64_peag_1877,BU124,dsp_25,33_bdx,14,4.508839,0.534071,5,9.675000,0.251558,3.465882,0.003946,19,nuit,False,True,0,0,0,0,0,0
3801,2024-12-01,64,M22,64_olt_3765,64_peag_1876,BU136,dsp_25,33_bdx,20,4.808550,0.810622,12,9.796875,0.289647,3.404412,0.207982,32,nuit,False,True,0,0,0,0,0,0


### Enregistrement des données preprocessed

In [65]:
df_imputed.to_parquet(
    "/content/drive/MyDrive/nexialog/data_preprocessed_FINAL.parquet",
    engine="pyarrow",
    index=False
)

# agrégation et feature engenering

In [ ]:
donnees_preprocessees = pd.read_parquet(chemin_donnees_preprocessees, engine="pyarrow").sort_values(by="date_hour")